# Aritficial Neural Network and SVM with Scikit-Learn

In [ ]:
from IPython.display import Image
%matplotlib inline

## Loading the Breast Cancer Wisconsin dataset

In [ ]:
"""
from sklearn import datasets
from sklearn.model_selection import train_test_split
bc = datasets.load_breast_cancer()
bc.keys()
X = bc.data
y = bc.target
print(X.shape, y.shape)
print(bc.feature_names)
print(bc.target_names)
"""

In [ ]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/'
                 'machine-learning-databases'
                 '/breast-cancer-wisconsin/wdbc.data', header=None)


df.head()

In [ ]:
df.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder

X = df.iloc[:, 2:].values
y = df.iloc[:, 1].values
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

In [ ]:
print(y.shape)
print(y)

Split Data into training and test sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, 
                     test_size=0.20,
                     random_state=2)

## Combining transformers and estimators in a pipeline

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline

In [ ]:
# Train the model with MLP
mlp = MLPClassifier(hidden_layer_sizes=(15,8), max_iter=1000, learning_rate_init=0.001, random_state=1)
mlp.fit(X_train, y_train)
print('Training Accuracy :', mlp.score(X_train, y_train))
print('Test Accuracy :', mlp.score(X_test, y_test))

In [ ]:
# Train the model with MLP after standardization
pipe_mlp = make_pipeline(StandardScaler(),
        MLPClassifier(hidden_layer_sizes=(15,8),  solver='lbfgs', 
                      max_iter=1000, random_state=1, learning_rate_init=0.001))

pipe_mlp.fit(X_train, y_train)
#y_pred = pipe_lr.predict(X_test)
print('Training Accuracy: %.3f' % pipe_mlp.score(X_train, y_train))
print('Test Accuracy: %.3f' % pipe_mlp.score(X_test, y_test))

## Support Vector Machines

In [ ]:
from sklearn.svm import SVC

In [ ]:
#train the model with SVM without standardization
svm = SVC(kernel='rbf', gamma=0.1, C=1, random_state=1)
svm.fit(X_train, y_train)
print('Training Accuracy :', svm.score(X_train, y_train))
print('Test Accuracy :', svm.score(X_test, y_test))

In [ ]:
#train the model with SVM after standardization
pipe_svm = make_pipeline(StandardScaler(), 
                         SVC(kernel='rbf', gamma=0.1, C=1.0, random_state=1))
pipe_svm.fit(X_train, y_train)
print('Training Accuracy: %.3f' % pipe_svm.score(X_train, y_train))
print('Test Accuracy: %.3f' % pipe_svm.score(X_test, y_test))

# Using k-fold cross validation to assess model performance

## K-fold cross-validation

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
    

kfold = StratifiedKFold(n_splits=5).split(X_train, y_train)

scores = []
for k, (train, test) in enumerate(kfold):
    pipe_mlp.fit(X_train[train], y_train[train])
    score = pipe_mlp.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

In [ ]:
pipe_mlp = make_pipeline(StandardScaler(),
        MLPClassifier(hidden_layer_sizes=(15,8),  solver='lbfgs', 
                      max_iter=1000, random_state=1, learning_rate_init=0.1))

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=pipe_mlp,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

# Debugging algorithms with learning curves

## Diagnosing bias and variance problems with learning curves

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LogisticRegression

pipe_lr = make_pipeline(StandardScaler(),
                        LogisticRegression(solver='lbfgs'))

train_sizes, train_scores, test_scores =\
                learning_curve(estimator=pipe_lr,
                               X=X_train,
                               y=y_train,
                               train_sizes=np.linspace(0.1, 1.0, 10),
                               cv=5,
                               scoring='accuracy')

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_mean,
         color='blue', marker='o',
         markersize=5, label='training accuracy')

plt.fill_between(train_sizes,
                 train_mean + train_std,
                 train_mean - train_std,
                 alpha=0.15, color='blue')

plt.plot(train_sizes, test_mean,
         color='green', linestyle='--',
         marker='s', markersize=5,
         label='validation accuracy')

plt.fill_between(train_sizes,
                 test_mean + test_std,
                 test_mean - test_std,
                 alpha=0.15, color='green')

plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.8, 1.03])
plt.tight_layout()
#plt.savefig('images/06_05.png', dpi=300)
plt.show()

## Addressing over- and underfitting with validation curves

In [ ]:
from sklearn.model_selection import validation_curve

pipe_lr = make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs', max_iter=1000))
#param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
param_range = np.logspace(-3, 2, 6)
train_scores, test_scores = validation_curve(
                estimator=pipe_lr, 
                X=X_train, 
                y=y_train, 
                param_name='logisticregression__C', 
                param_range=param_range,
                cv=5)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(param_range, train_mean, 
         color='blue', marker='o', 
         markersize=5, label='training accuracy')

plt.fill_between(param_range, train_mean + train_std,
                 train_mean - train_std, alpha=0.15,
                 color='blue')

plt.plot(param_range, test_mean, 
         color='green', linestyle='--', 
         marker='s', markersize=5, 
         label='validation accuracy')

plt.fill_between(param_range, 
                 test_mean + test_std,
                 test_mean - test_std, 
                 alpha=0.15, color='green')

plt.grid()
plt.xscale('log')
plt.legend(loc='lower right')
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.ylim([0.8, 1.0])
plt.tight_layout()
# plt.savefig('images/06_06.png', dpi=300)
plt.show()

# Fine-tuning machine learning models via grid search

## Tuning hyperparameters via grid search 

In [ ]:
from sklearn.model_selection import GridSearchCV

pipe_svc = make_pipeline(StandardScaler(),
                         SVC(random_state=1))

param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

param_grid = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf']}]



In [ ]:
gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=5,
                  n_jobs=-1)
gs = gs.fit(X_train, y_train)

print(gs.best_score_)
print(gs.best_params_)

In [ ]:
clf = gs.best_estimator_
clf.fit(X_train, y_train)
print('Training accuracy: %.3f' % clf.score(X_train, y_train))
print('Test accuracy: %.3f' % clf.score(X_test, y_test))

## Algorithm selection with nested cross-validation

In [ ]:
gs = GridSearchCV(estimator=pipe_svc,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=3)

scores = cross_val_score(gs, X_train, y_train, 
                         scoring='accuracy', 
                         cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

<br>
<br>

# Looking at different performance evaluation metrics

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = clf.predict(X_test)
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

## Optimizing the precision and recall of a classification model

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred))